In [1]:
from pyspark.sql import SparkSession
''
spark = (SparkSession
         .builder
         .appName('Analyzing the vocabulary of Pride and Prejudice')
         .getOrCreate())

In [2]:
import requests 
import tempfile

url = 'https://www.gutenberg.org/cache/epub/1342/pg1342.txt'
response = requests.get(url)
text = response.text

with tempfile.NamedTemporaryFile(delete=False, mode='w') as temp_file:
    temp_file.write(text)
    temp_path = temp_file.name

book = spark.read.text(temp_path)

In [15]:
import pyspark.sql.functions as F

results = (
    book
    .select(F.split(F.col('value'), ' ').alias('line'))
    .select(F.explode(F.col('line')).alias('word'))
    .select(F.lower(F.col('word')).alias('word'))
    .select(F.regexp_extract(F.col('word'), '[a-z]+',0).alias('word'))
    .filter(F.col('word') != '')
    .groupBy('word')
    .count()
    .orderBy(F.col('count').desc())
)

In [16]:
results.show()

+----+-----+
|word|count|
+----+-----+
| the| 4842|
|  to| 4399|
|  of| 3959|
| and| 3785|
| her| 2281|
|   i| 2105|
|   a| 2080|
|  in| 2039|
| was| 1877|
| she| 1744|
|that| 1639|
|  it| 1597|
| not| 1526|
| you| 1444|
|  he| 1359|
| his| 1302|
|  be| 1280|
|  as| 1240|
| had| 1186|
|with| 1148|
+----+-----+
only showing top 20 rows

